Principes
===

- multi-target (using all `resp`)
- weight also is a var
- features were generated
- using Time Cross Validation
- using Neptune to log

In [1]:
from models import create_encoder, create_model

import pandas as pd
import numpy as np
import tensorflow as tf
import random



from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from neptunecontrib.monitoring.keras import NeptuneMonitor

In [2]:
def get_threshhold(array, value):
    return np.median(array) + np.std(array)*value#- np.std(array)*0.35


def load_train_data(path_to_train_csv):
    # Load 50 rows to understand types of cols
    train_csv = pd.read_csv(path_to_train_csv, nrows=50) 
    # Finding float cols
    float_cols = [c for c in train_csv if train_csv[c].dtype == "float64"]
    # Float64 to Float16
    types = {c: 'float32' for c in float_cols}
    # Loading all rows
    train_csv = pd.read_csv(path_to_train_csv, engine='c', dtype=types) 
    
    
    resp = np.where(train_csv['resp'] > get_threshhold(train_csv['resp'].values, 0),    1, 0)
    resp_1 = np.where(train_csv['resp_1'] > get_threshhold(train_csv['resp_1'].values, 0.25),     1, 0)
    resp_2 = np.where(train_csv['resp_2'] > get_threshhold(train_csv['resp_2'].values, -0.25),    1, 0)
    resp_3 = np.where(train_csv['resp_3'] > get_threshhold(train_csv['resp_3'].values, -0.35),           1, 0)
    resp_4 = np.where(train_csv['resp_4'] > get_threshhold(train_csv['resp_4'].values, 0),           1, 0)
    

    train_csv['resp'] = resp_1
    train_csv['resp_1'] = resp_1
    train_csv['resp_2'] = resp_1
    train_csv['resp_3'] = resp_1
    train_csv['resp_4'] = resp_1
    
    
    return train_csv

In [3]:
def set_all_seeds(seed):
    np.random.seed(seed)
    random.seed(seed)
    tf.random.set_seed(seed)

In [4]:
train_csv = load_train_data('train_processed_sc.csv')
train_bool = True
evalution_bool = True

In [5]:
#Multitarget     # threshold- is tunable paramer, if threshold>0 - drops false postive rate
def spltter(train_csv, threshold=0):
#     test_csv = train_csv.sample(frac=0.2)
#     train_csv.drop(index=test_csv.index, inplace=True)
    
    dates = train_csv['date'].unique()
    test_dates = np.array([380, 168,  33,  94, 309, 348, 488, 476, 154,  37, 195,  30, 332,
       365, 363, 256, 320, 227, 219, 153, 424, 236,  22, 329, 165, 128,
       352, 187, 158,  45, 423, 150, 331,  64, 263, 296,  69, 144,  15,
       250, 426, 114, 223, 360, 269, 177, 208, 326, 429, 499, 214, 463,
       438, 178,  40, 121, 284, 258,   8, 143, 290, 321, 496, 215, 477,
       382,  58,  31,  54, 337, 497,  83,  39, 410, 455]) # #) #np.random.choice(dates, size=int(dates.size*0.15), replace=False, p=None)
    train_dates = np.setdiff1d(dates, test_dates)
    
    test_csv = train_csv[train_csv['date'].isin(test_dates)]
    train_csv = train_csv[train_csv['date'].isin(train_dates)]
    
    
    
    
    features = [c for c in train_csv.columns if 'feature' in c] + ['weight']
    resp_cols = ['resp_1', 'resp_2', 'resp_3', 'resp', 'resp_4'] 

    X_train = train_csv[features].values
    y_train = train_csv[resp_cols].values #np.stack([(train_csv[c] > 0).astype('int') for c in resp_cols]).T 
    
    X_test = test_csv[features].values
    y_test = test_csv[resp_cols].values# np.stack([(test_csv[c] > 0).astype('int') for c in resp_cols]).T 


    return X_train, y_train, X_test, y_test, test_dates

In [6]:
X_train, y_train, X_test, y_test, test_dates = spltter(train_csv)
del train_csv
X_train.shape

(2010418, 131)

In [7]:
test_dates

array([380, 168,  33,  94, 309, 348, 488, 476, 154,  37, 195,  30, 332,
       365, 363, 256, 320, 227, 219, 153, 424, 236,  22, 329, 165, 128,
       352, 187, 158,  45, 423, 150, 331,  64, 263, 296,  69, 144,  15,
       250, 426, 114, 223, 360, 269, 177, 208, 326, 429, 499, 214, 463,
       438, 178,  40, 121, 284, 258,   8, 143, 290, 321, 496, 215, 477,
       382,  58,  31,  54, 337, 497,  83,  39, 410, 455])

In [8]:
import neptune
api_token = 'eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vdWkubmVwdHVuZS5haSIsImFwaV91cmwiOiJodHRwczovL3VpLm5lcHR1bmUuYWkiLCJhcGlfa2V5IjoiYzhlNGMzYzYtZGU4Yi00OGY4LThiZTYtYzc0M2Q1NGU5MTRjIn0='
neptune.init(project_qualified_name='kopinhruk/nn-market',api_token=api_token)
neptune.create_experiment('proc_thresh', description='+0 select thresh, 0.4 pred thresh')

psutil is not installed. You will not be able to abort this experiment from the UI.
psutil is not installed. Hardware metrics will not be collected.


https://ui.neptune.ai/kopinhruk/nn-market/e/NNMAR-36


Experiment(NNMAR-36)

In [9]:
%%time
SEED = 123

if train_bool:
    X = np.concatenate((X_train, X_test))
    y = np.concatenate((y_train, y_test))
    set_all_seeds(SEED)

    autoencoder, encoder = create_encoder(X.shape[-1],y.shape[-1],noise=0.1)
    autoencoder.fit(X,(X,y),
                    epochs=1000,
                    batch_size=4096, 
                    validation_split=0.1,
                    callbacks=[
                        EarlyStopping('val_loss',patience=8,restore_best_weights=True),
                        #NeptuneMonitor()
                    ])
    #encoder.save_weights(f'./encoder_{SEED}.hdf5')
    encoder.trainable=False

    model = create_model(X.shape[-1],y.shape[-1],encoder)

    del X
    del y


    model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=100,batch_size=4096,
              callbacks=[EarlyStopping('val_loss',patience=15,restore_best_weights=True),
                         ReduceLROnPlateau('val_loss',patience=8),
                         NeptuneMonitor(prefix='clsf_')
                        ])

    model.save_weights(f'./weights/model_{SEED}.hdf5')


    del X_train, y_train#, X_test, y_test

    # Finetune
    #model.compile(Adam(hp.get('lr')/100),
                #  loss=BinaryCrossentropy(label_smoothing=10*hp.get('label_smoothing'))) #trying something with ls here
    #model.fit(X_test,y_test,epochs=3,batch_size=4096)
    #model.save_weights(f'./model_{SEED}_{fold}_finetune.hdf5')


Epoch 1/1000
526/526 [==============================] - 11s 13ms/step - loss: 0.9102 - decoded_loss: 0.3931 - label_output_loss: 0.5170 - val_loss: 0.6282 - val_decoded_loss: 0.1550 - val_label_output_loss: 0.4731
Epoch 2/1000
526/526 [==============================] - 6s 12ms/step - loss: 0.7143 - decoded_loss: 0.2233 - label_output_loss: 0.4911 - val_loss: 0.5991 - val_decoded_loss: 0.1266 - val_label_output_loss: 0.4725
Epoch 3/1000
526/526 [==============================] - 6s 12ms/step - loss: 0.6983 - decoded_loss: 0.2080 - label_output_loss: 0.4903 - val_loss: 0.5678 - val_decoded_loss: 0.0947 - val_label_output_loss: 0.4731
Epoch 4/1000
526/526 [==============================] - 6s 12ms/step - loss: 0.6913 - decoded_loss: 0.2020 - label_output_loss: 0.4892 - val_loss: 0.5581 - val_decoded_loss: 0.0864 - val_label_output_loss: 0.4717
Epoch 5/1000
526/526 [==============================] - 6s 12ms/step - loss: 0.6912 - decoded_loss: 0.2016 - label_output_loss: 0.4895 - val_loss: 

## Evalution / Submission 

In [10]:
if evalution_bool:
    # Creating model
    _, encoder = create_encoder(X_test.shape[-1],y_test.shape[-1],noise=0.1)
    encoder.trainable=False
    model = create_model(X_test.shape[-1],y_test.shape[-1],encoder)
    model.load_weights(f'./weights/model_{SEED}.hdf5')
    
    y_pred = model.predict(X_test, verbose=1, batch_size=4096) # n_samles x 5 (float)    
        
    print('Loading df')
    test_df = pd.read_csv('train.csv', usecols=['weight', 'resp', 'ts_id', 'date'], dtype={'weight': 'float32', 'resp': 'float32'})
    test_df = test_df[test_df['date'].isin(test_dates)]

93/93 [==============================] - 1s 5ms/step
Loading df


In [20]:
X_test.shape[-1],y_test.shape[-1]

(131, 5)

In [19]:
if evalution_bool:
    y_pred1 = np.mean(y_pred, axis=1) # n_samles x 1 (float)
   # y_pred1 = (y_pred[:,0]*0 + y_pred[:,1]*0 + y_pred[:,2]*0 + y_pred[:,3]*0 + y_pred[:,4]*1)/1
    y_pred_bool = np.where(y_pred1 > 0.435, 1, 0) # n_samles x 1 (int {0, 1})
    test_df['action'] = y_pred_bool
    test_df
    
    print('Calc metric')
    Pi = []
    for cur_date in np.unique(test_df['date']):
        temp_df = test_df[test_df['date'] == cur_date]
        value = sum(temp_df['weight'] * temp_df['resp'] * temp_df['action'])
        Pi.append(value)
    Pi = np.array(Pi)  
    t = np.sum(Pi) / np.sqrt(np.sum(Pi ** 2)) * np.sqrt(250 / np.unique(test_df['date']).size)
    u = min(max(t, 0), 6) * sum(Pi)
    print(u)
    neptune.log_metric('U', u)

Calc metric
1231.0537292364986


## Submission 

In [12]:
no_nan_df_mean = [ 1.11248678e-02,  4.16600823e-01,  3.55446428e-01,  8.58746283e-03,
        3.32153565e-03, -5.68061182e-03, -1.54546471e-02,  5.19358441e-02,
        2.70631295e-02,  2.69950360e-01,  1.72372252e-01,  9.73279849e-02,
        5.37668727e-02,  1.57676160e-01,  8.43710974e-02,  2.25111976e-01,
        1.37447536e-01,  1.21045269e-01,  1.13533348e-01,  2.88280666e-01,
        2.55277336e-01,  1.91205144e-01,  1.73040271e-01,  2.49782264e-01,
        2.28465229e-01,  2.86710739e-01,  2.57286400e-01,  1.35517642e-01,
        1.60874307e-01,  3.14388335e-01,  3.24808687e-01,  2.21792787e-01,
        2.46316716e-01,  3.03573936e-01,  3.20882916e-01,  3.33125502e-01,
        3.46933007e-01,  3.16174813e-02,  2.50911806e-02,  4.05390561e-02,
        5.01634404e-02,  3.89716804e-01,  4.39725071e-01,  4.10761237e-01,
        3.56510133e-01,  4.04844642e-01,  5.54753006e-01,  4.51396912e-01,
        5.94792664e-01,  6.41462445e-01,  5.99166214e-01,  4.92862254e-01,
        4.89415042e-03,  3.10924023e-01,  1.80537239e-01,  7.00669825e-01,
        7.03996062e-01,  5.74216425e-01,  4.84798193e-01,  5.82207561e-01,
        7.51079500e-01,  7.31717408e-01,  7.39191115e-01,  7.49898374e-01,
        8.44931602e-01,  8.55459690e-01,  7.95409739e-01,  9.03920829e-01,
        8.95861626e-01,  4.10086483e-01,  2.15152308e-01,  3.38279188e-01,
        2.02515069e-03, -3.67417000e-02, -4.26720502e-03, -2.14445405e-02,
       -3.52861769e-02, -1.12850480e-01, -9.14172083e-03, -3.85602303e-02,
       -4.00161138e-03, -1.62717886e-02, -3.70589420e-02, -1.12429135e-01,
        3.42513084e-01,  5.77558517e-01,  3.09788406e-01,  3.64835531e-01,
        5.02436042e-01,  6.54584467e-01,  3.51571441e-01,  5.31417191e-01,
        2.81628460e-01,  3.24677348e-01,  4.09467280e-01,  6.96833670e-01,
        3.42250884e-01,  5.84956169e-01,  2.91660517e-01,  3.87508065e-01,
        4.99385118e-01,  6.37722373e-01,  3.40197653e-01,  4.98174489e-01,
        2.95143634e-01,  3.34093809e-01,  4.30495739e-01,  6.96529508e-01,
        3.26292127e-01,  4.97213989e-01,  3.03535640e-01,  3.60349804e-01,
        4.41988647e-01,  7.07466841e-01,  3.46868783e-01,  4.29639369e-01,
        2.56859928e-01,  3.02295029e-01,  3.84930938e-01,  6.29942298e-01,
        2.24282220e-01, -1.12973236e-01,  2.63891965e-01,  1.73168648e-02,
        2.33980477e-01, -1.80340394e-01,  2.13665023e-01, -7.52404556e-02,
        2.46513456e-01, -1.51326917e-02,  3.09070754e+00]
no_nan_df_std = [0.99993837, 2.53277969, 2.43106151, 2.0240941 , 1.82004201,
       1.75321198, 1.67698264, 1.75748312, 2.01531267, 2.32869554,
       1.64647996, 1.54896557, 2.27313733, 2.11977959, 1.90222919,
       1.80233765, 2.13728499, 1.71327257, 2.22850299, 1.66572917,
       1.88361502, 2.3098278 , 1.74129856, 2.00890756, 2.4960227 ,
       2.00843859, 2.22804308, 1.51803434, 2.07495856, 1.68902409,
       2.03046441, 1.58326781, 2.38914442, 1.81675923, 1.95992076,
       2.3394475 , 2.21109748, 2.12443376, 2.21838903, 1.66017544,
       2.39466953, 1.97922707, 2.41636777, 2.33668399, 2.77381968,
       1.90095437, 3.05546713, 2.19827724, 3.21329427, 4.04049015,
       3.97951031, 2.8172338 , 1.87514794, 2.13836694, 1.80311501,
       6.52135086, 9.18529034, 6.50808477, 6.7414422 , 8.16410351,
       2.2825458 , 2.05775213, 2.23308778, 2.3178401 , 2.21779513,
       2.19248271, 1.76868069, 2.58077908, 2.52061629, 2.26492548,
       2.37525177, 1.79367197, 1.89814603, 2.23218226, 1.73965669,
       1.97060454, 2.28985643, 2.80546165, 2.29513144, 1.81658137,
       1.71284187, 2.24634171, 1.97238767, 2.59418797, 2.00102663,
       2.58498812, 1.84393692, 2.61183286, 2.65174198, 2.65999722,
       2.18634176, 2.72854304, 1.96498072, 2.15202546, 2.11638403,
       2.50706887, 2.14216948, 2.6429975 , 2.15489888, 2.03357697,
       2.29848719, 2.66446662, 2.45794606, 2.26143575, 1.74025154,
       2.03440189, 2.76839471, 2.48702765, 2.49868393, 2.61808228,
       1.81631255, 1.86282992, 2.61231852, 2.89367437, 2.21070409,
       2.31932569, 2.21543431, 2.43321896, 2.36629891, 2.0953939 ,
       1.81028557, 1.77445316, 2.0505228 , 1.79429746, 1.67098224,
       1.86403179, 2.4819932 , 1.52141416, 2.26780057, 1.55793428,
       7.72157001]
features = ['weight',
 'feature_0',
 'feature_1',
 'feature_2',
 'feature_3',
 'feature_4',
 'feature_5',
 'feature_6',
 'feature_7',
 'feature_8',
 'feature_9',
 'feature_10',
 'feature_11',
 'feature_12',
 'feature_13',
 'feature_14',
 'feature_15',
 'feature_16',
 'feature_17',
 'feature_18',
 'feature_19',
 'feature_20',
 'feature_21',
 'feature_22',
 'feature_23',
 'feature_24',
 'feature_25',
 'feature_26',
 'feature_27',
 'feature_28',
 'feature_29',
 'feature_30',
 'feature_31',
 'feature_32',
 'feature_33',
 'feature_34',
 'feature_35',
 'feature_36',
 'feature_37',
 'feature_38',
 'feature_39',
 'feature_40',
 'feature_41',
 'feature_42',
 'feature_43',
 'feature_44',
 'feature_45',
 'feature_46',
 'feature_47',
 'feature_48',
 'feature_49',
 'feature_50',
 'feature_51',
 'feature_52',
 'feature_53',
 'feature_54',
 'feature_55',
 'feature_56',
 'feature_57',
 'feature_58',
 'feature_59',
 'feature_60',
 'feature_61',
 'feature_62',
 'feature_63',
 'feature_64',
 'feature_65',
 'feature_66',
 'feature_67',
 'feature_68',
 'feature_69',
 'feature_70',
 'feature_71',
 'feature_72',
 'feature_73',
 'feature_74',
 'feature_75',
 'feature_76',
 'feature_77',
 'feature_78',
 'feature_79',
 'feature_80',
 'feature_81',
 'feature_82',
 'feature_83',
 'feature_84',
 'feature_85',
 'feature_86',
 'feature_87',
 'feature_88',
 'feature_89',
 'feature_90',
 'feature_91',
 'feature_92',
 'feature_93',
 'feature_94',
 'feature_95',
 'feature_96',
 'feature_97',
 'feature_98',
 'feature_99',
 'feature_100',
 'feature_101',
 'feature_102',
 'feature_103',
 'feature_104',
 'feature_105',
 'feature_106',
 'feature_107',
 'feature_108',
 'feature_109',
 'feature_110',
 'feature_111',
 'feature_112',
 'feature_113',
 'feature_114',
 'feature_115',
 'feature_116',
 'feature_117',
 'feature_118',
 'feature_119',
 'feature_120',
 'feature_121',
 'feature_122',
 'feature_123',
 'feature_124',
 'feature_125',
 'feature_126',
 'feature_127',
 'feature_128',
 'feature_129',
 'date',
 'ts_id']

In [13]:
if not train_bool:
    import janestreet
    env = janestreet.make_env()  

    th = 0.47
    
    # Creating model
    _, encoder = create_encoder(X_test.shape[-1],y_test.shape[-1],noise=0.1)
    encoder.trainable=False
    model = create_model(X_test.shape[-1],y_test.shape[-1],encoder)
    model.load_weights(f'./weights/model_{SEED}.hdf5')
    
    
    for (test_df, pred_df) in tqdm(env.iter_test()):     
        
        features = ["feature_%d" % i for i in range(130)] + ['weight']
        
        working_df = (test_df[features]-no_nan_df_mean)/no_nan_df_std
        working_df = working_df.fillna(0)


        y_pred = model.predict(working_df.values, verbose=1, batch_size=4096) # n_samles x 5 (float)    
        y_pred = np.mean(y_pred, axis=1) # n_samles x 1 (float)
        y_pred_bool = np.where(y_pred > th, 1, 0) # n_samles x 1 (int {0, 1})
        
        pred_df.action = y_pred_bool
        env.predict(pred_df)